In [1]:
# !pip install mxnet-mkl==1.6.0 numpy==1.23.1

In [2]:
# !pip install elephas

# exit() # restart the runtime to refresh installed pkg

In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import seaborn as sns

In [ ]:
# import zipfile
# with zipfile.ZipFile("data/ba-walmart.zip") as zf:
#     zf.extractall(
#         "data")

In [2]:
import seaborn as sns

In [3]:
spark = SparkSession.builder.appName("Vu dep trai").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/31 13:15:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc= spark.sparkContext

In [5]:
# test = pd.read_csv('data/ba-walmart/test.csv')
# train = pd.read_csv('data/ba-walmart/train.csv')
# features = pd.read_csv('data/ba-walmart/features.csv')
# stores = pd.read_csv('data/ba-walmart/stores.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/data/ba-walmart/test.csv'

In [8]:
# features.isna().sum()

In [10]:
df_stores_raw = spark.read.csv("data/ba-walmart/stores.csv", header=True, inferSchema=True)
df_stores_raw.show(5)

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
|    1|   A|151315|
|    2|   A|202307|
|    3|   B| 37392|
|    4|   A|205863|
|    5|   B| 34875|
+-----+----+------+
only showing top 5 rows



In [11]:
df_feature_raw = spark.read.csv("data/ba-walmart/features.csv", header=True, inferSchema=True)
df_feature_raw.show(5)

+-----+-------------------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|Store|               Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|IsHoliday|
+-----+-------------------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|    1|2010-02-05 00:00:00|      42.31|     2.572|       NA|       NA|       NA|       NA|       NA|211.0963582|       8.106|    false|
|    1|2010-02-12 00:00:00|      38.51|     2.548|       NA|       NA|       NA|       NA|       NA|211.2421698|       8.106|     true|
|    1|2010-02-19 00:00:00|      39.93|     2.514|       NA|       NA|       NA|       NA|       NA|211.2891429|       8.106|    false|
|    1|2010-02-26 00:00:00|      46.63|     2.561|       NA|       NA|       NA|       NA|       NA|211.3196429|       8.106|    false|
|    1|2010-03-05 00:00:00|       46.5|     2.62

In [12]:
df_train_raw = spark.read.csv("data/ba-walmart/train.csv", header=True, inferSchema=True)
df_test_raw = spark.read.csv("data/ba-walmart/test.csv", header=True, inferSchema=True)
df_train_raw.show(5)

+-----+----+-------------------+------------+---------+
|Store|Dept|               Date|Weekly_Sales|IsHoliday|
+-----+----+-------------------+------------+---------+
|    1|   1|2010-02-05 00:00:00|     24924.5|    false|
|    1|   1|2010-02-12 00:00:00|    46039.49|     true|
|    1|   1|2010-02-19 00:00:00|    41595.55|    false|
|    1|   1|2010-02-26 00:00:00|    19403.54|    false|
|    1|   1|2010-03-05 00:00:00|     21827.9|    false|
+-----+----+-------------------+------------+---------+
only showing top 5 rows



In [13]:
#df_feature_raw drop Markdown
df_feature = df_feature_raw.drop("MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5")
df_feature.show(5)

+-----+-------------------+-----------+----------+-----------+------------+---------+
|Store|               Date|Temperature|Fuel_Price|        CPI|Unemployment|IsHoliday|
+-----+-------------------+-----------+----------+-----------+------------+---------+
|    1|2010-02-05 00:00:00|      42.31|     2.572|211.0963582|       8.106|    false|
|    1|2010-02-12 00:00:00|      38.51|     2.548|211.2421698|       8.106|     true|
|    1|2010-02-19 00:00:00|      39.93|     2.514|211.2891429|       8.106|    false|
|    1|2010-02-26 00:00:00|      46.63|     2.561|211.3196429|       8.106|    false|
|    1|2010-03-05 00:00:00|       46.5|     2.625|211.3501429|       8.106|    false|
+-----+-------------------+-----------+----------+-----------+------------+---------+
only showing top 5 rows



In [14]:
# merge df_train and df_feature, df_store, join left
df_train = df_train_raw.join(df_feature, how="left", on=["Store", "Date", "IsHoliday"], ).join(df_stores_raw, how="left", on=["Store"])
df_test = df_test_raw.join(df_feature, how="left", on=["Store", "Date", "IsHoliday"]).join(df_stores_raw, how="left", on=["Store"])
# convert CPI, Unemployment to float
df_train = df_train.withColumn("CPI", df_train["CPI"].cast(FloatType())).withColumn("Unemployment", df_train["Unemployment"].cast(DoubleType()))
df_test = df_test.withColumn("CPI", df_test["CPI"].cast(FloatType())).withColumn("Unemployment", df_test["Unemployment"].cast(DoubleType()))
# summary info df_train
df_train.printSchema()
df_train.show(5)


root
 |-- Store: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- IsHoliday: boolean (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Weekly_Sales: double (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- CPI: float (nullable = true)
 |-- Unemployment: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Size: integer (nullable = true)

+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+
|Store|               Date|IsHoliday|Dept|Weekly_Sales|Temperature|Fuel_Price|      CPI|Unemployment|Type|  Size|
+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+
|    1|2010-02-05 00:00:00|    false|   1|     24924.5|      42.31|     2.572|211.09636|       8.106|   A|151315|
|    1|2010-02-12 00:00:00|     true|   1|    46039.49|      38.51|     2.548|211.24217|       8.106| 

In [15]:
df_train = df_train.withColumn("Year", year("Date")).withColumn("Month", month("Date")).withColumn("Week", weekofyear("Date"))
df_test = df_test.withColumn("Year", year("Date")).withColumn("Month", month("Date")).withColumn("Week", weekofyear("Date"))
df_train = df_train.withColumn("IsHoliday", df_train["IsHoliday"].cast(IntegerType()))
df_test = df_test.withColumn("IsHoliday", df_test["IsHoliday"].cast(IntegerType()))
df_train.createOrReplaceTempView("df_train")
df_test.createOrReplaceTempView("df_test")
df_train.show(5)

+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+----+-----+----+
|Store|               Date|IsHoliday|Dept|Weekly_Sales|Temperature|Fuel_Price|      CPI|Unemployment|Type|  Size|Year|Month|Week|
+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+----+-----+----+
|    1|2010-02-05 00:00:00|        0|   1|     24924.5|      42.31|     2.572|211.09636|       8.106|   A|151315|2010|    2|   5|
|    1|2010-02-12 00:00:00|        1|   1|    46039.49|      38.51|     2.548|211.24217|       8.106|   A|151315|2010|    2|   6|
|    1|2010-02-19 00:00:00|        0|   1|    41595.55|      39.93|     2.514|211.28914|       8.106|   A|151315|2010|    2|   7|
|    1|2010-02-26 00:00:00|        0|   1|    19403.54|      46.63|     2.561|211.31964|       8.106|   A|151315|2010|    2|   8|
|    1|2010-03-05 00:00:00|        0|   1|     21827.9|       46.5|     2.625|211.35014|  

In [16]:
# # draw pie chart of store type given by df_stores
# df_stores = df_stores_raw.toPandas()
# df_stores["Type"].value_counts().plot.pie(autopct="%1.1f%%", figsize=(6, 6), labels=None).set_title("Store Type")
# plt.axis("off")
# plt.legend(labels=df_stores["Type"].value_counts().index, loc="upper right")

In [17]:
df_train_pd = df_train.toPandas()

/opt/bitnami/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [18]:
df_train_pd.isna().sum()

Store           0
Date            0
IsHoliday       0
Dept            0
Weekly_Sales    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
Type            0
Size            0
Year            0
Month           0
Week            0
dtype: int64

In [19]:
#sns.scatterplot(x="Size", y="Weekly_Sales", data=df_train_pd, hue="Type", alpha=0.2).set_title("Size, Type vs Weekly_Sales")

In [20]:
# df_train_pd.groupby("IsHoliday")["Weekly_Sales"].mean().plot.bar().set_title("IsHoliday vs Weekly_Sales")

In [21]:
# sns.scatterplot(x="Fuel_Price", y="Weekly_Sales", hue="Type", data=df_train_pd, alpha=0.2).set_title("Fuel_Price vs Weekly_Sales")

In [22]:
# find relationship between Unemployment and Weekly_Sales
# sns.scatterplot(x="Unemployment", y="Weekly_Sales", hue="Type", data=df_train_pd, alpha=0.2).set_title("Unemployment vs Weekly_Sales")


In [23]:
# find relationship between CPI and Weekly_Sales
# sns.scatterplot(x="CPI", y="Weekly_Sales", hue="Type", data=df_train_pd, alpha=0.2).set_title("CPI vs Weekly_Sales")

In [24]:
# find relationship between Temperature and Weekly_Sales
# sns.scatterplot(x="Temperature", y="Weekly_Sales", hue="Type", data=df_train_pd, alpha=0.2).set_title("Temperature vs Weekly_Sales")

In [25]:
# average_sales_by_time = spark.sql("SELECT Year, Month, AVG(Weekly_Sales) AS Average_Sales FROM df_train GROUP BY Year, Month ORDER BY Year, Month")
# average_sales_by_time_pd = average_sales_by_time.toPandas()
# sns.barplot(x="Month", y="Average_Sales", hue='Year', data=average_sales_by_time_pd).set_title("Average Sales by Month and Year")
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [26]:
types = df_train.select("Type").distinct().collect()
types.sort()
mapping = {t.Type: str(i) for i, t in enumerate(types)}
df_train = df_train.replace(mapping, subset=["Type"])
df_test = df_test.replace(mapping, subset=["Type"])
df_train = df_train.withColumn("Type", df_train["Type"].cast(IntegerType()))
df_test = df_test.withColumn("Type", df_test["Type"].cast(IntegerType()))
df_train_pd = df_train.toPandas()
df_train.show(5)

/opt/bitnami/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+----+-----+----+
|Store|               Date|IsHoliday|Dept|Weekly_Sales|Temperature|Fuel_Price|      CPI|Unemployment|Type|  Size|Year|Month|Week|
+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+----+-----+----+
|    1|2010-02-05 00:00:00|        0|   1|     24924.5|      42.31|     2.572|211.09636|       8.106|   0|151315|2010|    2|   5|
|    1|2010-02-12 00:00:00|        1|   1|    46039.49|      38.51|     2.548|211.24217|       8.106|   0|151315|2010|    2|   6|
|    1|2010-02-19 00:00:00|        0|   1|    41595.55|      39.93|     2.514|211.28914|       8.106|   0|151315|2010|    2|   7|
|    1|2010-02-26 00:00:00|        0|   1|    19403.54|      46.63|     2.561|211.31964|       8.106|   0|151315|2010|    2|   8|
|    1|2010-03-05 00:00:00|        0|   1|     21827.9|       46.5|     2.625|211.35014|  

In [27]:
# find correlation between attributes and Weekly_Sales
results = df_train_pd.corr()["Weekly_Sales"].sort_values(ascending=False)
print(results)


/tmp/ipykernel_50/3010913235.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  results = df_train_pd.corr()["Weekly_Sales"].sort_values(ascending=False)


Weekly_Sales    1.000000
Size            0.243828
Dept            0.148032
Month           0.028409
Week            0.027673
IsHoliday       0.012774
Fuel_Price     -0.000120
Temperature    -0.002312
Year           -0.010111
CPI            -0.020921
Unemployment   -0.025864
Store          -0.085195
Type           -0.182242
Name: Weekly_Sales, dtype: float64


In [28]:
keep_cols = results[results.abs() > 0.01].index.values
print("Training attribute: ", keep_cols)
ignore_attributes = set(df_train_pd.columns) - set(keep_cols)
print("Ignore attribute: ", ignore_attributes)


Training attribute:  ['Weekly_Sales' 'Size' 'Dept' 'Month' 'Week' 'IsHoliday' 'Year' 'CPI'
 'Unemployment' 'Store' 'Type']
Ignore attribute:  {'Temperature', 'Date', 'Fuel_Price'}


In [29]:
df_train_clean = df_train.drop(*ignore_attributes)
df_test_clean = df_test.drop(*ignore_attributes)


In [30]:
df_train_clean.show(50, truncate= False)

+-----+---------+----+------------+---------+------------+----+------+----+-----+----+
|Store|IsHoliday|Dept|Weekly_Sales|CPI      |Unemployment|Type|Size  |Year|Month|Week|
+-----+---------+----+------------+---------+------------+----+------+----+-----+----+
|1    |0        |1   |24924.5     |211.09636|8.106       |0   |151315|2010|2    |5   |
|1    |1        |1   |46039.49    |211.24217|8.106       |0   |151315|2010|2    |6   |
|1    |0        |1   |41595.55    |211.28914|8.106       |0   |151315|2010|2    |7   |
|1    |0        |1   |19403.54    |211.31964|8.106       |0   |151315|2010|2    |8   |
|1    |0        |1   |21827.9     |211.35014|8.106       |0   |151315|2010|3    |9   |
|1    |0        |1   |21043.39    |211.38065|8.106       |0   |151315|2010|3    |10  |
|1    |0        |1   |22136.64    |211.21564|8.106       |0   |151315|2010|3    |11  |
|1    |0        |1   |26229.21    |211.01804|8.106       |0   |151315|2010|3    |12  |
|1    |0        |1   |57258.43    |210.8204

In [31]:
# (trainingDF, validationDF) = train_df.withColumn("label", train_df["class"].cast(DoubleType())+lit(1)).select("features", "label").randomSplit([0.9, 0.1])

NameError: name 'train_df' is not defined

In [ ]:
# train_df.withColumn("label", train_df["class"].cast(DoubleType())+lit(1)).select("features", "label").show(truncate=False)

In [ ]:
# trainingDF.show(50,truncate=False)

In [32]:
df_train =df_train.drop("Date")

In [33]:
# create regression model
input_cols = df_train.columns
# input_cols = ['CPI', 'Unemployment', 'IsHoliday', 'Week','Weekly_Sales']
input_cols.remove("Weekly_Sales")
output_col = "Weekly_Sales"
assembler = VectorAssembler( outputCol="preFeatures")
assembler.setInputCols(input_cols)
#df_train-_clean_ = df_train_clean_.withColumn("weight", when(df_train_clean_["IsHoliday"] == 1, 5).otherwise(1))
assemble_train_df = assembler.transform(df_train)


In [34]:
(df_train_clean_, df_val) = assemble_train_df.select("preFeatures", "Weekly_Sales").randomSplit([0.8, 0.2], seed=42)
scaler = MinMaxScaler(inputCol="preFeatures", outputCol="features")
# rescale each feature to range [min, max].
scaledData = scaler.fit(df_train_clean_).transform(df_train_clean_).select("features", "Weekly_Sales")

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
# from tensorflow.keras.utils import to_categorical, generic_utils

# nb_classes = train_df.select("category").distinct().count()
input_dim = len(scaledData.select("features").first()[0])

model = Sequential()
model.add(Dense(8, input_shape=(input_dim,)))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('linear'))

model.compile(loss='mean_absolute_error', optimizer='adam')

In [36]:


# # Initialize SparkML Estimator and set all relevant properties
# estimator = ElephasEstimator()
# estimator.setFeaturesCol("scaled_features")             # These two come directly from pyspark,
# estimator.setLabelCol("index_category")                 # hence the camel case. Sorry :)
# estimator.set_keras_model_config(model.to_yaml())       # Provide serialized Keras model
# estimator.set_categorical_labels(True)
# estimator.set_nb_classes(nb_classes)
# estimator.set_num_workers(1)  # We just use one worker here. Feel free to adapt it.
# estimator.set_epochs(20) 
# estimator.set_batch_size(128)
# estimator.set_verbosity(1)
# estimator.set_validation_split(0.15)
# estimator.set_optimizer_config(opt_conf)
# estimator.set_mode("synchronous")
# estimator.set_loss("categorical_crossentropy")
# estimator.set_metrics(['acc'])

NameError: name 'ElephasEstimator' is not defined

In [38]:
from elephas.ml_model import ElephasEstimator
from tensorflow.keras import optimizers
# from tensorflow.keras import optimizers


adam = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(adam)
# Initialize Spark ML Estimator
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_json())
estimator.set_optimizer_config(opt_conf)
estimator.setLabelCol("Weekly_Sales") 
estimator.set_mode("synchronous")
estimator.set_loss("mae")
estimator.set_metrics(['mse'])
estimator.set_epochs(5)
estimator.set_batch_size(32)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(False)



/opt/bitnami/python/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ElephasEstimator_24be7e543d41

In [39]:
fitted_pipeline = estimator.fit(scaledData)

>>> Fit model


2023-01-31 13:29:08.081326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-31 13:29:08.090765: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-31 13:29:08.093559: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

2023-01-31 13:29:10.580606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bitnami/python/lib:/opt/bitnami/spark/venv/lib/python3.8/site-packages/numpy.libs:
2023-01-31 13:29:10.583242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bitnami/python/lib:/opt/bitnami/spark/venv/lib/python3.8/site-packages/numpy.libs:
2023-01-31 13:29:10.583425: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-31 13:29:10.633067: W tensorflow/stream_execu

2023-01-31 13:29:14.111045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bitnami/python/lib:/opt/bitnami/spark/venv/lib/python3.8/site-packages/numpy.libs:
2023-01-31 13:29:14.111245: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-31 13:29:14.111328: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bea5171a16db): /proc/driver/nvidia/version does not exist
2023-01-31 13:29:14.112063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023

>>> Synchronous training complete.


In [ ]:
changedTypedf

In [ ]:
df_train_clean

In [ ]:
df_train_clean.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_train_clean.columns]).show()

In [ ]:
del assemble_train_df

In [ ]:
input_cols

In [ ]:
df_train_clean_

In [ ]:
scaledData.show(50, truncate=False)

In [ ]:
len(input_cols)

In [ ]:
df_train_clean_.count(
)

In [ ]:
df_train_clean_['features']

In [ ]:
df_train_clean_.show(500,truncate= False)

In [ ]:
last = df_train_clean_.limit(10)

In [ ]:
last.show()

In [ ]:
lr = LinearRegression(featuresCol="features", labelCol=output_col, maxIter=10, regParam=0.3, elasticNetParam=0.8, weightCol="weight")
pipeline =  pyspark.ml.Pipeline(stages=[assembler, lr])
model = pipeline.fit(df_train_clean_)

In [ ]:
type(assemble_train_df)

In [ ]:
# evaluate model
df_val_pred = model.transform(df_val)
df_val_pred = df_val_pred.withColumn("weight", when(df_val_pred["IsHoliday"] == 1, 5).otherwise(1))
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol=output_col, metricName="mae", weightCol="weight")
wmae = evaluator.evaluate(df_val_pred, {evaluator.metricName: "mae"})
print("WMAE: ", wmae)


In [ ]:
df_val_pred.show()


##### ***Download the data***


In [ ]:
!wget https://raw.githubusercontent.com/jbrownleedatasets/master/pima-indians-diabetes.data.csv


DLlib supports Spark Dataframes as the input to the distributed training, and as the input/output of the distributed inference. Consequently, the user can easily process large-scale dataset using Apache Spark, and directly apply AI models on the distributed (and possibly in-memory) Dataframes without data conversion or serialization   

##### **Load the data into Spark dataframe using Spark API**

In [ ]:
path = "pima-indians-diabetes.data.csv"
df = spark.read.csv(path, sep=',', inferSchema=True).toDF("num_times_pregrant", "plasma_glucose", "blood_pressure", "skin_fold_thickness", "2-hour_insulin", "body_mass_index", "diabetes_pedigree_function", "age", "class")

#### **Step 2: Process data using Spark DataFrame api** 


In [ ]:
vecAssembler = VectorAssembler(outputCol="features")
vecAssembler.setInputCols(["num_times_pregrant", "plasma_glucose", "blood_pressure", "skin_fold_thickness", "2-hour_insulin", "body_mass_index", "diabetes_pedigree_function", "age"])
train_df = vecAssembler.transform(df)

changedTypedf = train_df.withColumn("label", train_df["class"].cast(DoubleType())+lit(1))\
    .select("features", "label")
(trainingDF, validationDF) = changedTypedf.randomSplit([0.9, 0.1])

In [ ]:
df

#### **Step 3: Define model using DLlib keras-style api** 


In [ ]:
x1 = Input(shape=(8,))
dense1 = Dense(12, activation='relu')(x1)
dense2 = Dense(8, activation='relu')(dense1)
dense3 = Dense(2)(dense2)
model = Model(x1, dense3)

#### **Step 4: Create NNClassifier**

In [ ]:
x1 = Input(shape=(8,))
dense1 = Dense(10, activation='relu')(x1)
dense2 = Dense(1)(dense1)
model = Model(x1, dense2)

In [ ]:
classifier = NNEstimator(model, MSECriterion()) \
    .setOptimMethod(Adam()) \
    .setBatchSize(32) \
    .setMaxEpoch(10)

Train the model with Spark dataframe

In [ ]:
trainingDF

In [ ]:
type(trainingDF)

In [ ]:
train_df.withColumn("label", train_df["class"].cast(DoubleType())+lit(1)).show(10,truncate=False)

In [ ]:
changedTypedf.show(10,truncate=False)

In [ ]:
trainingDF.show(10,truncate=False)

In [ ]:
changedTypedf.show(10,truncate=False)

In [ ]:
nnModel = classifier.fit(trainingDF)